In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../") # go to parent dir
import gym
from gym import spaces
import numpy as np
import sciunit
import scipy

## Env for decision making

In [3]:
from src.ldmunit.bandit import BanditEnv, BanditAssociateEnv
from src.ldmunit.models import decision_making, associative_learning
from src.ldmunit.models.utils import loglike, train_with_obs, simulate

## Decision Marking

In [4]:
n_action = 2
n_obs = 3
n_trials = 100
env = BanditEnv([0.3, 0.7, 0.1])

In [5]:
# print("testing for random responding")
paras = {'bias': 0.8, 'action_bias': 0}

model = decision_making.RandomRespondModel(n_action, n_obs, paras)
stimuli, rewards, actions = simulate(env, model, n_trials)
a = loglike(model, stimuli, rewards, actions)
print("The log-likelihood: {:10.5}".format(a))

opt = train_with_obs(model, stimuli, rewards, actions)
if opt.success:
    print(opt.x)
else:
    print(opt)

The log-likelihood:    -66.676
[0.8 0. ]


In [6]:
print("testing for noisy-win-stay-lose-shift")
paras = {'epsilon': 0.5}

model = decision_making.NWSLSModel(n_action, n_obs, paras)
stimuli, rewards, actions = simulate(env, model, n_trials)
a = loglike(model, stimuli, rewards, actions)
print("The log-likelihood: {:10.5}".format(a))

opt = train_with_obs(model, stimuli, rewards, actions)
if opt.success:
    print(opt.x)
else:
    print(opt)

testing for noisy-win-stay-lose-shift
The log-likelihood:    -55.135
[0.5]


In [7]:
print("testing for RWCK")
paras = {'w0': 0.1, 'alpha': 0.5, 'alpha_c': 0.5, 'beta': 0.5, 'beta_c': 0.5}

model = decision_making.RWCKModel(n_action, n_obs, paras=paras)
stimuli, rewards, actions = simulate(env, model, 100)
fixed = {'w0': 0.123}
a = loglike(model, stimuli, rewards, actions)
print("The log-likelihood: {:10.5}".format(a))

opt = train_with_obs(model, stimuli, rewards, actions, fixed)
if opt.success:
    print(opt.x)
else:
    print(opt)

testing for RWCK
The log-likelihood:    -67.689
[0.123 0.5   0.5   0.5   0.5  ]


In [8]:
from src.ldmunit.models.utils import MultiMeta, multi_from_single

param_list = [{'epsilon': 0.5}] * 10
MultiNWSLS = multi_from_single(decision_making.NWSLSModel, 'MultiNWSLS')
models = MultiNWSLS(param_list, n_action=2, n_obs=5)

print(MultiNWSLS.__bases__)
print(models.act(2,2))

(<class 'src.ldmunit.models.decision_making.base.DADO'>,)
0



## Associate learning

In [9]:
print("testing for RwNormModel")
paras = {'w0': 0.123, 'alpha': 0.5, 'sigma': 0.5, 'b0': 0.5, 'b1': 0.5}

model = associative_learning.RwNormModel(n_obs, paras)
env = BanditAssociateEnv([0.3, 0.7, 0.8])
stimuli, rewards, actions = simulate(env, model, n_trials)
a = loglike(model, stimuli, rewards, actions)
print("The log-likelihood: {:10.5}".format(a))
if opt.success:
    print(opt.x)
else:
    print(opt)

testing for RwNormModel
The log-likelihood:    -78.051
[0.123 0.5   0.5   0.5   0.5  ]


In [10]:
print("testing for KrwNormModel")
paras = {'w0': 0.1, 'alpha': 0.5, 'sigma': 0.5, 'b0': 0.5, 'b1': 0.5, 'logSigmaWInit': 0.23, 'logTauSq': 0.32,
        'logSigmaRSq': 0.34}

model = associative_learning.KrwNormModel(n_obs, paras)
stimuli, rewards, actions = simulate(env, model, n_trials)
a = loglike(model, stimuli, rewards, actions)
print("The log-likelihood: {:10.5}".format(a))

opt = train_with_obs(model, stimuli, rewards, actions)
if opt.success:
    print(opt.x)
else:
    print(opt)

testing for KrwNormModel
The log-likelihood:    -73.026
[0.1  0.5  0.5  0.5  0.5  0.23 0.32 0.34]


In [11]:
print("testing for lsspd")
paras = {'w0': 0.1, 'alpha': 0.5, 'sigma': 0.5, 'b0': 0.5, 'b1': 0.5, 'eta': 0.23, 'kappa': 0.32,
        'mix_coef': 0.34}

model = associative_learning.LSSPDModel(n_obs, paras)
stimuli, rewards, actions = simulate(env, model, n_trials)
a = loglike(model, stimuli, rewards, actions)
print("The log-likelihood: {:10.5}".format(a))

opt = train_with_obs(model, stimuli, rewards, actions)
if opt.success:
    print(opt.x)
else:
    print(opt)

testing for lsspd
The log-likelihood:    -73.684
[0.1  0.5  0.5  0.5  0.5  0.23 0.32 0.34]


In [12]:
print("testing for beta binomial")
paras = {'b0': 0.5, 'b1': 0.56, 'mix_coef': 0.34, 'sigma': 0.99}
model = associative_learning.BetaBinomialModel(3, paras)
stimuli, rewards, actions = simulate(env, model, n_trials, [[0.3, 0.7, 0.1], [0.99, 0.01, 0.5]])
a = loglike(model, stimuli, rewards, actions)
print("The log-likelihood: {:10.5}".format(a))

opt = train_with_obs(model, stimuli, rewards, actions)
if opt.success:
    print(opt.x)
else:
    print(opt)

testing for beta binomial
The log-likelihood:    -166.05
[0.5  0.56 0.34 0.99]


In [13]:
print("testing for beta binomial")
paras = {'b0': 0.5, 'b1': 0.56, 'mix_coef': 0.34, 'sigma': 0.99}
model = associative_learning.BetaBinomialModel(3, paras)
stimuli, rewards, actions = simulate([env, env], model, n_trials)
a = loglike(model, stimuli, rewards, actions)
print("The log-likelihood: {:10.5}".format(a))

opt = train_with_obs(model, stimuli, rewards, actions)
if opt.success:
    print(opt.x)
else:
    print(opt)

testing for beta binomial
The log-likelihood:    -137.64
[0.5  0.56 0.34 0.99]
